In [1]:
import math #basic python math library, used for square rooting and getting pi
import numpy as np #common library, source of numpy arrays which are easier to manipulate as vectors than your average python list

from scipy.constants import e, m_e, epsilon_0 
#physics constants I need, the charge of a proton or electron (e), the mass of an electron (m_e), and epsilon naught

import matplotlib.pyplot as plt #for plotting data
import matplotlib.animation as animation #to create animation
from IPython.display import HTML

In [2]:
class Charge:

    def __init__(self, x, y, charge):
        self.x_pos = float(x)
        self.y_pos = float(y)
        self.charge = charge

    def get_position(self):
        return np.array([self.x_pos, self.y_pos])

    def get_charge(self):
        return self.charge

In [3]:
class Electron:

    def __init__(self, x, y):
        self.position = np.array([float(x), float(y)])
        self.charge = -e
        self.velocity = np.array([0.0, 0.0])
        self.acceleration = np.array([0.0, 0.0])

    def get_position(self):
        return self.position

    def get_force(self, charges):
        force = np.array([0.0, 0.0])
        for charge in charges:
            r = math.sqrt((charge.get_position() - self.get_position())[0] ** 2 + (charge.get_position() - self.get_position())[1] ** 2)
            force -= (charge.get_position() - self.get_position()) * charge.get_charge() * self.charge / 4 / math.pi / epsilon_0 / (r ** 3)
        self.acceleration = force / m_e / 1000
   
    def get_acceleration(self):
        return self.acceleration

    def update_velocity(self):
        self.velocity = self.acceleration

    def update_position(self):
        self.position += self.velocity
        print(self.position)

In [4]:
def electron_update(charges, electron):

    electron.get_force(charges)
    electron.update_velocity()
    electron.update_position()

In [8]:
i = 0

def gen(charges, electron):
    global i
    sep = True
    while sep:
        for charge in charges:
            if abs(electron.get_position()[0] - charge.get_position()[0]) < 0.05 and abs(electron.get_position()[1] - charge.get_position()[1]) < 0.05:
                sep = False
                break
            if np.array_equal(electron.get_acceleration(), [0.0,0.0]):
                print(electron.get_acceleration())
                sep = False
            if sep == True:
                i += 1
                yield i

In [6]:
def create_animation(charges, electron):

    fig, ax = plt.subplots()
    
    scatter = ax.scatter([c.get_position()[0] for c in charges], [c.get_position()[1] for c in charges], c="r", label="charges")
    scatter = ax.scatter(electron.get_position()[0], electron.get_position()[1], c="g", label="electron")

    ax.set_xlim([-10, 10])
    ax.set_ylim([-10, 10])
    ax.legend()

    def update(frame):

        electron_update(charges, electron)
        scatter.set_offsets(electron.get_position())
        return scatter

    anim = animation.FuncAnimation(fig=fig, func=update, frames=gen(charges, electron), interval=20, save_count=10)
    plt.close()

    return HTML(anim.to_jshtml())

In [9]:
c1 = Charge(2, 0, e)
c2 = Charge(-2,0,e)
charges = [c1]
electron = Electron(1, 2)
create_animation(charges, electron)

[0. 0.]


C:\Users\dbart\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\matplotlib\animation.py:1741: UserWarning: Can not start iterating the frames for the initial draw. This can be caused by passing in a 0 length sequence for *frames*.

If you passed *frames* as a generator it may be exhausted due to a previous display or save.
  warnings.warn(
